In [122]:

from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta


DB_PATH = r"C:\python\dataOnline\anaslo_02\anaslo_02.db"

spreadSheet_ids = {
    "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    "アスカ狭山店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    }

# 検索キーワードよりホール名取得
SEARCH_WORD = "EXA FIRST"
SPREADSHEET_ID = spreadSheet_ids[SEARCH_WORD]

# スプレッドシート認証設定
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
jsonf = r"C:\python\dataOnline\anaslo_02\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key(SPREADSHEET_ID)

# # Table name 取得
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = cursor.fetchall()
print(tables)

cursor.execute(
    "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + SEARCH_WORD + "%",)
)
results = cursor.fetchall()

# 結果表示
if results:
    print(f"🔍 '{SEARCH_WORD}' を含むホール名の検索結果:")
    for hall_id, hall_name in results:
        print(f" - hall_name: {hall_name}, hall_id: {hall_id}")
else:
    print(f"❌ '{SEARCH_WORD}' を含むホール名は見つかりませんでした。")

query = """
-- 出玉データにホール名と機種名を結合して取得
SELECT
    r.*, 
    h.name AS hall_name,     -- ホール名を追加
    m.name AS model_name     -- 機種名を追加
FROM results r
JOIN halls h ON r.hall_id = h.hall_id  -- ホールと結合
JOIN models m ON r.model_id = m.model_id  -- 機種と結合
WHERE h.name = ?  -- 指定ホールのみ
  AND m.name LIKE '%ジャグラー%'  -- ジャグラー系機種に限定
ORDER BY r.date DESC, r.unit_no ASC;
"""

df = pd.read_sql_query(query, conn, params=(hall_name,))
df["date"] = pd.to_datetime(df["date"])
df.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
df = df[["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]]
df["BB_rate"] = (df["game"] / df["BB"]).round(1)
df["RB_rate"] = (df["game"] / df["RB"]).round(1)
df["Total_rate"] = (df["game"] / (df["BB"] + df["RB"])).round(1)
df["day"] = df["date"].dt.day
df["month"] = df["date"].dt.month
df["weekday"] = df["date"].dt.weekday

conn.close()

df.head()
df.date.nunique()

[('halls',), ('models',), ('prefectures',), ('results',), ('sqlite_sequence',)]
🔍 'EXA FIRST' を含むホール名の検索結果:
 - hall_name: EXA FIRST, hall_id: 108


341

## HOLE_ANALYSIS

In [ ]:
# 日ごとにモデル別で表示
SHEET_NAME = "HOLE_ANALYSIS"
df_tmp = df.copy()
# メダル
medals = df_tmp.pivot_table(
    index=["model_name"],
    columns="day",
    values="medals",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
# RB＿RATE
game = df_tmp.pivot_table(
    index=["model_name"],
    columns="day",
    values="game",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
rb = df_tmp.pivot_table(
    index=["model_name"],
    columns="day",
    values="RB",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
rb_rate = (game / rb).round(1)
rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
rb_rate = rb_rate.replace([np.inf, -np.inf], np.nan)
rb_rate = rb_rate.fillna("")
game.columns = pd.MultiIndex.from_product([["GAME"], game.columns])
# WRITE
sheet = spreadsheet.worksheet(SHEET_NAME)
sheet.clear()
set_with_dataframe(sheet, medals, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)
next_row = len(existing) + 4  # +2: ヘッダー + 次の空行
set_with_dataframe(sheet, rb_rate, row=next_row, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)
next_row = len(existing) + 6  # +2: ヘッダー + 次の空行
set_with_dataframe(sheet, game, row=next_row, include_index=True)

## UNIT_ANALYSIS

In [ ]:
# マシン別メダル
SHEET_NAME = "UNIT_ANALYSIS"
MACHINE_NAME = "マイジャグラーV"
df_tmp = df[df["model_name"] == MACHINE_NAME].copy()
df_tmp = df.copy()
# メダル数
medals = df_tmp.pivot_table(
    index=["model_name", "unit_no"],
    columns="day",
    values="medals",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
# medals["rank"] = medals["Total"].rank(method="min", ascending=False).astype(int)
medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
medals.shape
# RB_rate 作成
game = df_tmp.pivot_table(
    index=["model_name", "unit_no"],
    columns="day",
    values="game",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
rb = df_tmp.pivot_table(
    index=["model_name", "unit_no"],
    columns="day",
    values="RB",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
rb_rate = (game / rb).round(1)
# rb_rate["rank"] = rb_rate["Total"].rank(method="min", ascending=True).astype(int)
rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
rb_rate = rb_rate.replace([np.inf, -np.inf], np.nan)
rb_rate = rb_rate.fillna(0)
rb_rate.shape

# 回転数
games = df_tmp.pivot_table(
    index=["model_name", "unit_no"],
    columns="day",
    values="game",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
# games["rank"] = games["Total"].rank(method="min", ascending=False).astype(int)
games.columns = pd.MultiIndex.from_product([["GAMES"], games.columns])
games.head()

next_row = 1
spreadsheet = client.open_by_key(SPREADSHEET_ID)
sheet = spreadsheet.worksheet(SHEET_NAME)
sheet.clear()
set_with_dataframe(sheet, medals, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)
next_row = len(existing) + 4  # +2: ヘッダー + 次の空行
set_with_dataframe(sheet, rb_rate, row=next_row, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)
next_row = len(existing) + 6  # +2: ヘッダー + 次の空行
set_with_dataframe(sheet, games, row=next_row, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)

## DAY_DETAIL_N

In [ ]:
# 日付指定
DAY = 11
SHEET_NAME = f"DAY_DETAIL_{DAY}"

next_row = 1
spreadsheet = client.open_by_key(SPREADSHEET_ID)
sheet = spreadsheet.worksheet(SHEET_NAME)
sheet.clear()

models = [
    "マイジャグラーV",
    # "ゴーゴージャグラー3",
    # "アイムジャグラーEX-TP",
    # "ファンキージャグラー2",
]

for model in models:

    df_tmp = df[(df["day"] == DAY) & (df["model_name"] == model)].copy()
    df_tmp["date"] = df_tmp["date"].dt.strftime("%Y-%m-%d")

    # メダル数
    medals = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="medals",
        aggfunc="sum",
        margins=True,
        margins_name="Total",
    )
    medals.drop(labels="Total", level=0, inplace=True)
    medals["Rank"] = medals["Total"].rank(method="min", ascending=False).astype(int)
    medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
    print(f"{model}_{medals.shape}")

    # RB_RATE
    game = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="game",
        aggfunc="sum",
        margins=True,
        margins_name="Total",
    )
    rb = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="RB",
        aggfunc="sum",
        margins=True,
        margins_name="Total",
    )
    
    rb_rate = (game / rb).round(1)
    medals.drop(labels="Total", level=0, inplace=True)
    rb_rate["rank"] = rb_rate["Total"].rank(method="min", ascending=True).astype(int)
    rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
    rb_rate = rb_rate.replace([np.inf, -np.inf], np.nan)
    rb_rate = rb_rate.fillna(0)
    print(f"{model}_{rb_rate.shape}")

    # スプレッドシート追記
    set_with_dataframe(sheet, medals, row=next_row, include_index=True)
    existing = get_as_dataframe(sheet, evaluate_formulas=True)
    next_row += medals.shape[0] + 5  # 追記行
    set_with_dataframe(sheet, rb_rate, row=next_row, include_index=True)
    existing = get_as_dataframe(sheet, evaluate_formulas=True)
    next_row += medals.shape[0] + 5  # 追記行

マイジャグラーV_(81, 13)
マイジャグラーV_(82, 13)


## 過去三日のメダル数と翌日のRB確率を表示 → 転記

In [154]:
def medals_and_rb_rate(machine_name, year=2025, month=4, day=5, period=3):
    base_date = datetime.date(year, month, day)
    start_date = base_date + datetime.timedelta(days=-1)
    end_date = base_date + datetime.timedelta(days=-period)
    df_tmp = df[
        (df["model_name"] == machine_name)
        & (df["date"].dt.date <= start_date)
        & (df["date"].dt.date >= end_date)
    ]
    df_tmp["date"].unique()

    # メダル
    medals = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="medals",
        aggfunc="sum",
        margins=True,
        margins_name="Total",
    )
    medals.drop(labels="Total", level=0, inplace=True)
    medals["rank"] = medals["Total"].rank(method="min", ascending=True).astype(int)
    medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
    medals.tail()

    # RB_RATE
    df_tmp = df[(df["model_name"] == machine_name) & (df["date"].dt.date == base_date)]
    game = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="game",
        aggfunc="sum",
        # margins=True,
        # margins_name="Total",
    )
    bb = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="BB",
        aggfunc="sum",
        # margins=True,
        # margins_name="Total",
    )
    rb = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="RB",
        aggfunc="sum",
        # margins=True,
        # margins_name="Total",
    )
    rb_rate = (game / rb).round(1)
    # rb_rate["judge"] = rb_rate.iloc[:, 0] <= 268.6
    rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
    rb_rate.head()
    
    total_rate = (game / (bb+rb)).round(1)
    total_rate.columns = pd.MultiIndex.from_product([["TOTAL_RATE"], total_rate.columns])

    game.columns = pd.MultiIndex.from_product([["GAME"], game.columns])

    # データを表示（長い場合は全表示）
    pd.set_option("display.max_rows", None)
    pd.set_option("display.max_columns", None)
    medal_rb_rate = pd.concat([medals, game, total_rate, rb_rate], axis=1)
    medal_rb_rate = medal_rb_rate.replace([np.inf, -np.inf], np.nan)
    medal_rb_rate = medal_rb_rate.fillna(0)

    medal_rb_rate = medal_rb_rate[medal_rb_rate["MEDALS"]["rank"] <= 10]

    return medal_rb_rate


# N_DAYS_AGO
MACHINE_NAME = "マイジャグラーV"
YEAR, MONTH, DAY = 2025, 3, 12
period = 3

medal_rb_rate = medals_and_rb_rate(MACHINE_NAME, 2025, 3, 12, period)
medal_rb_rate.head()

MEDALS                      \
date               2025-03-09 00:00:00 2025-03-10 00:00:00   
model_name unit_no                                           
マイジャグラーV   1001                  -1806                -562   
           1005                   -350                -915   
           1011                  -2003                -562   
           1020                   -703               -1288   
           1034                  -3141                  97   

                                                                 GAME  \
date               2025-03-11 00:00:00 Total rank 2025-03-12 00:00:00   
model_name unit_no                                                      
マイジャグラーV   1001                    238 -2130    7                1887   
           1005                   -626 -1891   10                 918   
           1011                   -809 -3374    3                7710   
           1020                   -524 -2515    6                7761   
           1034                     65 -2979    4                 916   

                            TOTAL_RATE             RB_RATE  
date               2025-03-12 00:00:00 2025-03-12 00:00:00  
model_name unit_no                                          
マイジャグラーV   1001                  145.2               377.4  
           1005                  183.6               918.0  
           1011                  122.4               233.6  
           1020                  138.6               352.8  
           1034                  305.3               916.0

In [ ]:
# スプレッドシート追記
DAY = 11
SHEET_NAME = f"DAY{DAY}x3DAYS_AGO"

models = [
    "マイジャグラーV",
    "ゴーゴージャグラー3",
    "アイムジャグラーEX-TP"
    ]

spreadsheet = client.open_by_key(SPREADSHEET_ID)
sheet = spreadsheet.worksheet(SHEET_NAME)
sheet.clear()

today = datetime.date.today()
base = datetime.date(today.year, today.month, DAY)
dates = [base - relativedelta(months=i+1) for i in range(7)]

next_row = 1
for model in models:
    for d in dates:
        medal_rb_rate = medals_and_rb_rate(model, d.year, d.month, d.day, period)
        # display(medal_rb_rate)
        set_with_dataframe(sheet, medal_rb_rate, row=next_row, include_index=True)
        existing = get_as_dataframe(sheet, evaluate_formulas=True)
        next_row += medal_rb_rate.shape[0] + 5

## N日前までのデータを表示

In [153]:
def rb_rate_n_days_ago(start_date, end_date, model_name):
    # RB_RATE
    df_tmp = df[
        (df["model_name"] == model_name)
        & (df["date"].dt.date <= start_date)
        & (df["date"].dt.date >= end_date)
    ].copy()

    medals = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="medals",
        aggfunc="sum",
        margins=True,
        margins_name="Total",
    )
    medals.drop(labels="Total", level=0, inplace=True)
    medals["Rank"] = medals["Total"].rank(method="min", ascending=True).astype(int)
    medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])

    return medals


today = datetime.date.today()
start_date = today + datetime.timedelta(days=-1)
end_date = today + datetime.timedelta(days=-3)
rb_rate_n_days_ago(start_date, end_date, model).head()

MEDALS                      \
date                2025-04-07 00:00:00 2025-04-08 00:00:00   
model_name  unit_no                                           
ファンキージャグラー2 1051                    368                -632   
            1052                    947                -232   
            1053                   -485                -179   
            1054                   -479                -756   
            1055                   1194                 191   

                                                    
date                2025-04-09 00:00:00 Total Rank  
model_name  unit_no                                 
ファンキージャグラー2 1051                   -124  -388    3  
            1052                   1441  2156    8  
            1053                   2444  1780    7  
            1054                   -403 -1638    1  
            1055                   1932  3317    9

In [ ]:


SHEET_NAME = "nDAYS_AGO"

today = datetime.date.today()
start_date = today + datetime.timedelta(days=-1)
end_date = today + datetime.timedelta(days=-3)

models = [
    "マイジャグラーV",
    "ゴーゴージャグラー3",
    "アイムジャグラーEX-TP",
    "ファンキージャグラー2",
]

sheet = spreadsheet.worksheet(SHEET_NAME)
sheet.clear()

next_row = 1
for model in models:
    medals = rb_rate_n_days_ago(start_date, end_date, model)
    set_with_dataframe(sheet, medals, row=next_row, include_index=True)
    existing = get_as_dataframe(sheet, evaluate_formulas=True)
    next_row += medals.shape[0] + 5

### 
- 3月5日の過去1日間の出玉ワースト10 → 336以下は5台
- 3月5日の過去2日間の出玉ワースト10 → 336以下は3台
- 3月5日の過去3日間の出玉ワースト10 → 336以下は7台
- 3月5日の過去4日間の出玉ワースト10 → 336以下は6台
### 
- 1月12日の過去1日間の出玉ワースト10 → 336以下は5台
- 1月12日の過去2日間の出玉ワースト10 → 336以下は5台
- 1月12日の過去3日間の出玉ワースト10 → 336以下は5台
- 1月12日の過去4日間の出玉ワースト10 → 336以下は4台
- 1月12日の過去5日間の出玉ワースト10 → 336以下は3台
### 
- 2月12日の過去1日間の出玉ワースト10 → 336以下は3台
- 2月12日の過去2日間の出玉ワースト10 → 336以下は4台
- 2月12日の過去3日間の出玉ワースト10 → 336以下は5台
- 2月12日の過去4日間の出玉ワースト10 → 336以下は4台
- 2月12日の過去5日間の出玉ワースト10 → 336以下は5台
###
- 3月12日の過去1日間の出玉ワースト10 → 336以下は0台
- 3月12日の過去2日間の出玉ワースト10 → 336以下は1台
- 3月12日の過去3日間の出玉ワースト10 → 336以下は3台
- 3月12日の過去4日間の出玉ワースト10 → 336以下は4台
- 3月12日の過去5日間の出玉ワースト10 → 336以下は3台
- 3月12日の過去6日間の出玉ワースト10 → 336以下は3台
- 3月12日の過去7日間の出玉ワースト10 → 336以下は4台